In [1]:
%matplotlib inline
import os
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import LineString
import folium

In [2]:
def add_markers(mapobj, gdf):
    coords = []
    for i, row in gdf.iterrows():
        coords.append([row.geometry.y, row.geometry.x])
    for coord in coords:
        folium.CircleMarker(location = coord,
                            radius = 2.5, 
                            fill = True,
                            fill_color = '#F50057',
                            fill_opacity = 0.75,
                            color = 'whitesmoke',
                            weight = 0.5).add_to(mapobj)
    return mapobj

gps = 'timemanager_test_data.shp'
tracks = gpd.GeoDataFrame.from_file(gps)
tracks = tracks[['track_se_1', 'ele', 'time', 'geometry']].copy()

f = folium.Figure(height = 400)
m = folium.Map([35.792809, -78.675724], zoom_start = 15, tiles='Cartodb dark_matter')
m.add_to(f)

add_markers(m, tracks)

In [3]:
tracks.head()

,track_se_1,ele,time,geometry
0,0,125.7,2017/12/29 21:42:18.000,POINT Z (-78.668663 35.787979 125.7)
1,1,125.6,2017/12/29 21:42:23.000,POINT Z (-78.668841 35.787985 125.6)
2,2,125.4,2017/12/29 21:42:25.000,POINT Z (-78.66897 35.788028 125.4)
3,3,125.2,2017/12/29 21:42:27.000,POINT Z (-78.669096 35.788059 125.2)
4,4,125.1,2017/12/29 21:42:29.000,POINT Z (-78.669222 35.788086 125.1)


In [14]:
tracks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 4 columns):
track_se_1    163 non-null int64
ele           163 non-null float64
time          163 non-null object
geometry      163 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 5.2+ KB


In [35]:
geom0 = gdf.loc[0, tracks]
geom1 = gdf.loc[1, tracks]

# Create LineString from coordinates
start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
line = LineString([start, end])
length = line.length

TypeError: unhashable type: 'Point'

In [32]:
def make_lines(gdf, df_out, i, elevation = 'ele', time = 'time', geometry = 'geometry'):
    # Get track coordinates
    geom0 = gdf.loc[i, geometry]
    geom1 = gdf.loc[i + 1, geometry]
    
    # Create LineString from coordinates
    start, end = [(geom0.x, geom0.y), (geom1.x, geom1.y)]
    line = LineString([start, end])
    length
    
    # Calculate change in elevation
    elevation_change = gdf.loc[i + 1, elevation] - gdf.loc[i, elevation]
    
    # Calculate time betweent segments
    time_change = pd.to_datetime(gdf.loc[i + 1, time], infer_datetime_format=True) - pd.to_datetime(gdf.loc[i, time], infer_datetime_format=True)
    time_change_seconds = time_change.seconds
    
    # Create a DataFrame to hold record
    data = {'id': i,
            'elevation_change': elevation_change,
            'time_change': time_change_seconds,
            'geometry': [line]}
    df_line = pd.DataFrame(data, columns = ['id', 'elevation_change', 'time_change','geometry'])
    
    # Add record DataFrame of compiled records
    df_out = pd.concat([df_out, df_line])
    return df_out

In [34]:
# Create an 
df = pd.DataFrame(columns = ['id', 'elevation_change', 'time_change', 'geometry'])

x = 0
while x < len(tracks) - 1:
    df_input = make_lines(tracks, df, x)
    df = df_input
    x = x + 1

df.head()

,id,elevation_change,time_change,geometry
0,0,-0.1,5,"LINESTRING (-78.668663 35.787979, -78.668841 3..."
0,1,-0.2,2,"LINESTRING (-78.668841 35.787985, -78.66897 35..."
0,2,-0.2,2,"LINESTRING (-78.66897 35.788028, -78.669096 35..."
0,3,-0.1,2,"LINESTRING (-78.669096 35.788059, -78.669222 3..."
0,4,-0.1,2,"LINESTRING (-78.669222 35.788086, -78.669372 3..."


In [23]:
crs = {'init': 'epsg:4326'}
gdf = GeoDataFrame(df, crs=crs)

folium.GeoJson(gdf).add_to(m)
m

In [45]:
gdf = gdf.to_crs(epsg = 3857)
gdf['distance_traveled'] = gdf.length
gdf['m_per_sec'] = gdf['distance_traveled'] / gdf['time_change']
gdf = gdf.to_crs(epsg = 4326)
gdf

,elevation_change,geometry,id,time_change,distance_traveled,m_per_sec
0,-0.1,"LINESTRING (-78.668663 35.78797899999999, -78....",0,5.0,19.831969,3.966394
0,-0.2,LINESTRING (-78.66884099999997 35.787985000000...,1,2.0,15.525351,7.762675
0,-0.2,"LINESTRING (-78.66897 35.788028, -78.669096 35...",2,2.0,14.657204,7.328602
0,-0.1,"LINESTRING (-78.669096 35.78805899999998, -78....",3,2.0,14.507397,7.253698
0,-0.1,"LINESTRING (-78.66922199999999 35.788086, -78....",4,2.0,17.165622,8.582811
0,-0.3,"LINESTRING (-78.669372 35.78811499999996, -78....",5,2.0,18.879556,9.439778
0,-0.2,"LINESTRING (-78.669535 35.78815300000001, -78....",6,2.0,18.425153,9.212577
0,-0.1,"LINESTRING (-78.669693 35.78819299999999, -78....",7,2.0,19.357039,9.678519
0,-0.1,"LINESTRING (-78.669859 35.788235, -78.670034 3...",8,2.0,19.911177,9.955589
0,-0.1,"LINESTRING (-78.670034 35.78826499999999, -78....",9,2.0,18.420414,9.210207


In [43]:
def export_gdf(gdf, filename):
    try:
        os.remove(filename)
    except OSError:
        pass
    gdf.to_file('output.geojson', driver = "GeoJSON")

In [46]:
export_gdf(gdf, 'output.geojson')